In [38]:
#Question2
#Step 1: Importing Libraries
import cvxpy as cp
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [5]:
#Step 2: Importing the dataset
train = pd.read_csv('train.csv',header=None )
test = pd.read_csv('test.csv',header=None)

In [6]:
train.shape

(8500, 201)

In [7]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,0.0,-0.36,-0.91,-0.99,-0.57,-1.38,-1.54,-1.64,1.29,0.65,...,1.15,-0.05,-0.09,0.02,1.75,1.58,0.12,0.30,2.44,-1.26
1,1.0,-1.40,-1.90,0.09,0.29,-0.30,-1.30,1.13,-2.38,-1.16,...,0.48,0.24,-0.16,-0.48,-0.02,-0.35,-0.27,-0.20,-0.92,-0.46
2,1.0,-0.43,1.45,-0.68,-1.58,0.32,-0.14,0.23,-1.01,-0.39,...,-0.94,0.11,-1.30,-0.24,0.74,0.88,1.37,0.12,0.01,-0.56
3,1.0,-0.76,0.30,-0.57,-0.33,-1.50,1.84,1.37,0.23,0.66,...,-0.42,0.06,-1.05,0.35,-0.24,-0.69,1.31,-0.18,-1.54,-1.70
4,0.0,-0.76,1.36,0.00,-1.44,-1.27,-0.76,-1.42,-0.58,0.11,...,-0.44,1.45,-1.18,-1.13,-0.14,0.04,0.33,1.20,-0.81,-1.16


In [8]:
#Step 3: Data preprocessing: Split train.csv into training & validation
y_train = np.array(train[0])
y_train.shape

(8500,)

In [9]:
x_train = np.array(train.drop([0], axis=1))
x_train

array([[-0.36, -0.91, -0.99, ...,  0.3 ,  2.44, -1.26],
       [-1.4 , -1.9 ,  0.09, ..., -0.2 , -0.92, -0.46],
       [-0.43,  1.45, -0.68, ...,  0.12,  0.01, -0.56],
       ...,
       [ 1.18,  0.78, -0.02, ..., -0.63,  0.29, -0.96],
       [-0.55,  1.16, -0.56, ...,  2.97, -2.66, -1.9 ],
       [ 0.8 , -0.97, -0.15, ..., -1.58,  0.35,  1.06]])

In [ ]:
#Step 4: Training the Algorithm

In [10]:
x_train.shape #8499 data points, 201 features

(8500, 200)

In [11]:
x_train.shape[1]

200

In [55]:
# Training data: SVM soft margin (primal form)
# Data is not linearly separable
def svm_train_primal(data_train,label_train,regularisation_para_C):
    
    #we initialize w & b
   
    n = x_train.shape[0] #n is number of data points
    d = x_train.shape[1] #d is number of dimensions
    w = cp.Variable(d)   
    slack = cp.Variable(n)
    b = cp.Variable(1)
    P = np.eye(d, dtype = int)
    y = label_train
    x = data_train
    regularisation_para_C = 100
    
    objective_func = cp.Minimize((1/2) * cp.quad_form(w, P) +  regularisation_para_C/n* cp.sum(slack))
    constraints = [cp.multiply(y,((x @ w) + b)) >= 1 - slack, slack >= 0]                       
    
    prob = cp.Problem(objective_func, constraints)
    prob.solve()
    
    return w, b

In [59]:
w, b = svm_train_primal(x_train,y_train,100)

In [60]:
np.sum(w.value)

-1.7269591083835655e-11

In [61]:
print(b.value)

[1.60000006]


In [26]:
#QUESTION 3: Training alg Soft-margin Linear SVM from its dual form
def svm_train_dual(data_train, label_train, regularisation_para_C):
    
    n = x_train.shape[0] #n is number of data points
    d = x_train.shape[1] #d is number of dimensions
    alpha = cp.Variable(n) 
    y = label_train   # y(labels) & x (features)
    x = data_train


                    #how to put double sum(Rewrite according to lec slides)
    objective_func = cp.Maximize(sum(alpha) - 1/2* cp.quad_form(cp.multiply(alpha,y), x @ x.T))
    constraints = [0 <= alpha, alpha <= regularisation_para_C/n, alpha.T @ y == 0]
    
    prob = cp.Problem(objective_func, constraints)
    prob.solve()

    return alpha

In [27]:
alpha = svm_train_dual(x_train,y_train,100)

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
QuadForm(var85053 @ [0. 1. 1. ... 1. 1. 0.], [[239.9546  -9.2648  -9.9393 ...   8.39    -7.7384   3.45  ]
 [ -9.2648 280.2659 -45.0966 ...  54.0715  54.8081  23.103 ]
 [ -9.9393 -45.0966 259.6921 ... -21.2601 -31.26   -14.5716]
 ...
 [  8.39    54.0715 -21.2601 ... 246.1333  35.1174  -2.3419]
 [ -7.7384  54.8081 -31.26   ...  35.1174 243.7156 -35.6727]
 [  3.45    23.103  -14.5716 ...  -2.3419 -35.6727 249.9757]])

In [39]:
#QUESTION 4 : Write code to obtain the primal problem solution 𝐰∗,𝑏∗ from its dual solution 𝛼∗

def get_primal_solution(data_train, label_train, alpha):
    
    n = x_train.shape[0] 
    d = x_train.shape[1]
    y = label_train
    x = data_train
    mult = np.multiply(alpha,y) #elementwise mult
    mult_x = (mult * x.T).T
    w_star = np.sum(mult_x, axis=0) #w = summation of all samples on the wrong side
    b_star = y[0] - w_star.T @ x[0,:]
  
    return w_star, b_star;


In [40]:
get_primal_solution(x_train, y_train, np.random.normal(size=x_train.shape[0]))

(array([  54.22727091,  132.43552078, -101.28575669,  -63.60228886,
        -140.01611267,  177.6452937 ,  -48.10807555,  193.23509519,
        -210.92300812,  -70.27300312,  -78.00353995,   74.71208459,
          77.00388832,   30.2906333 ,   90.7317305 , -125.35227328,
          -2.38376248,  110.19689116,    9.8275155 ,  -47.39634545,
         -52.75151345,   61.11730223,  140.27933635,  146.59057023,
         -39.02373063,   -7.86477195,  240.09704295,  -36.20752325,
         -81.3810232 ,  301.95646061, -162.1987518 ,  110.31224237,
         100.11236671,  151.6231751 ,  -62.19730093,   31.96974612,
         104.50222276, -145.02385964,   36.61443758, -101.4597854 ,
         121.40168306,  -92.77851083,  -94.590493  ,    3.03857672,
         -20.83724092, -189.22350725,  212.36337612,   21.43148136,
         -42.32634181, -134.59633925, -136.72117965,  102.564753  ,
         -14.5151266 ,  157.52116503,   -6.46506489, -122.42826504,
        -147.00385359, -118.21720544,   59.43243

In [29]:
y_train[y_train == 0] = -1
y_train


array([-1.,  1.,  1., ...,  1.,  1., -1.])

In [ ]:
#QUESTION 5: Find the support vectors from primal problem solutions

def get_support_vectors_primal(data_train, label_train, regularisation_para_C):    
    
    n = x_train.shape[0] 
    d = x_train.shape[1]
    alpha_star = cp.Variable() #dual variable ??
    
    y = label_train
    x = data_train
#Need to implement two conditions to find support vectors:
    if(y(w_star.T @ x + b_star) < 1 && slack != 0 && alpha_star = regularisation_para_C):
        #what do we return??
         return x,y #Returning support vectors
    else if(0 < a_star < regularisation_para_C && slack 0 &&  y(w_star.T + b_star) = 1)
         return x,y 


In [42]:
#QUESTION 6: Find the support vectors from dual problem solutions

def get_support_vectors_dual(data_train, label_train, alpha): 
    # Can use Question 3 to find alpha. Check if alpha is > 0
    support_vecs = []
    for i in range(len(alpha)):
        if alpha[i] > 0:
            support_vecs.append(i)

    return support_vecs 

In [43]:
get_support_vectors_dual(x_train,y_train,np.random.normal(size=x_train.shape[0]))

[1,
 2,
 3,
 4,
 5,
 6,
 10,
 12,
 13,
 14,
 15,
 16,
 17,
 19,
 20,
 21,
 24,
 26,
 28,
 29,
 30,
 31,
 33,
 34,
 36,
 40,
 42,
 44,
 49,
 50,
 56,
 57,
 58,
 59,
 61,
 65,
 67,
 68,
 72,
 73,
 75,
 76,
 79,
 81,
 83,
 85,
 86,
 87,
 90,
 91,
 92,
 97,
 98,
 99,
 103,
 104,
 106,
 109,
 110,
 111,
 113,
 116,
 117,
 118,
 123,
 124,
 125,
 126,
 127,
 128,
 130,
 133,
 135,
 138,
 140,
 141,
 142,
 143,
 144,
 148,
 150,
 151,
 153,
 155,
 158,
 159,
 161,
 163,
 166,
 168,
 169,
 170,
 171,
 172,
 173,
 175,
 178,
 180,
 182,
 184,
 185,
 186,
 188,
 189,
 190,
 193,
 195,
 198,
 199,
 200,
 204,
 206,
 207,
 209,
 210,
 212,
 214,
 216,
 217,
 218,
 221,
 224,
 230,
 232,
 236,
 237,
 238,
 239,
 241,
 242,
 243,
 244,
 248,
 250,
 252,
 253,
 254,
 255,
 256,
 258,
 259,
 261,
 266,
 267,
 268,
 269,
 271,
 272,
 273,
 274,
 275,
 276,
 278,
 281,
 283,
 284,
 286,
 287,
 288,
 290,
 295,
 296,
 298,
 300,
 301,
 303,
 307,
 309,
 310,
 311,
 312,
 314,
 316,
 319,
 320,
 324,
 326

In [44]:
#Question 7: Write code to choose C by using the validation set.
#np.random.choice(np.arange(x_train.shape[0]), 10, replace=False)

array([5961, 5751, 5909, 1372, 7152,  648, 1939, 4923,  270, 3800])

In [47]:
#Creating the validation set the validation set
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [48]:
X_val.shape #Verifying that the validation set has been created properly

(1700, 200)

In [49]:
X_train.shape

(6800, 200)

In [50]:
x_train.shape

(8500, 200)

In [63]:
def question7(X_val, Y_val):
        
    range_C = 2**np.arange(-10.0,12.0,2.0)
    n = x_train.shape[0] #n is number of data points
    d = x_train.shape[1] #d is number of dimensions
    w = cp.Variable(d)   
    slack = cp.Variable(n)
    b = cp.Variable(1)
    P = np.eye(d, dtype = int)
    y = label_train
    x = data_train
    
    for i in range(len(range_C)):
        if(cp.multiply(y,((x @ w) + b) > 0)
            return range_C[i]
        else if(cp.multiply(y,((x @ w) + b) < 0)
            return range_C[i]

SyntaxError: invalid syntax (<ipython-input-63-b4a15492388d>, line 15)

In [ ]:
#Question 8: 